### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

from pandas import json_normalize



In [2]:
import os

# Print current working directory
print("Current Working Directory:", os.getcwd())

# List files in the current directory
print("Files in Current Directory:", os.listdir())

Current Working Directory: /Users/katiekavanagh/Desktop/M6_Starter_Code
Files in Current Directory: ['.DS_Store', 'output', 'collected_data.csv', '.env', '.ipynb_checkpoints', 'retrieve_movie_data.ipynb']


In [8]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


In [9]:
import os

# Print current working directory
print("Current Working Directory:", os.getcwd())

# List files in the current directory
print("Files in Current Directory:", os.listdir())


Current Working Directory: /Users/katiekavanagh/Desktop/M6_Starter_Code
Files in Current Directory: ['.DS_Store', 'output', 'collected_data.csv', '.env', '.ipynb_checkpoints', 'retrieve_movie_data.ipynb']


### Access the New York Times API

In [10]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"



In [11]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    query_url = f"{url}fq={filter_query}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}&page={page}"
    response = requests.get(query_url)
    time.sleep(12)  # Stay within API rate limits
    
    try:
        reviews = response.json()
        if "response" in reviews:
            docs = reviews["response"]["docs"]
            reviews_list.extend(docs)
            print(f"Page {page} completed.")
        else:
            print(f"Page {page} has no response key.")
            print("Response content:", json.dumps(reviews, indent=4))
            break
    except Exception as e:
        print(f"Failed to get data from page {page}: {e}")
        print("Response content:", response.text)
        break
# Preview first five results
print(json.dumps(reviews_list[:5], indent=4))

    

Page 0 completed.
Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Page 6 completed.
Page 7 completed.
Page 8 completed.
Page 9 completed.
Page 10 completed.
Page 11 completed.
Page 12 completed.
Page 13 completed.
Page 14 completed.
Page 15 completed.
Page 16 completed.
Page 17 completed.
Page 18 completed.
Page 19 completed.
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
       

In [12]:
# reviews_list is converted to a Pandas DataFrame using json_normalize() 
df_reviews = json_normalize(reviews_list)

# The title is extracted from the "headline.main" column and is saved in a new column "title" 
df_reviews['title'] = df_reviews['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

# The "keywords" column is correctly converted to string data using the supplied extract_keywords function
def extract_keywords(keywords):
    return ", ".join([kw['value'] for kw in keywords])

df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)

# A list called titles is created from the "title" column using to_list() 
titles = df_reviews['title'].to_list()

# Display the DataFrame
df_reviews.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"Movies, Kapur, Shekhar, James, Lily, Azmi, Sha...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"Movies, You Can Live Forever (Movie), Slutsky,...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"Movies, A Tourist's Guide to Love (Movie), Tsu...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"Movies, Zlotowski, Rebecca, Other People's Chi...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [13]:
#PART 2 ACCESS TMDB  ***********************************************************************************

# Base URL for TMDB API
tmdb_search_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_movie_url = "https://api.themoviedb.org/3/movie/"
tmdb_key_string = "&api_key=" + tmdb_api_key

# Create an empty list to store the movie details
tmdb_movies_list = []

# Create a variable called request_counter and initialize it with the value of 1
request_counter = 1



In [15]:
# Loop through the titles list created from the movie reviews DataFrame
for title in titles:
    # Construct the search query URL
    search_query_url = f"{tmdb_search_url}{title}{tmdb_key_string}"
    
    # Make a GET request to search for the movie by title
    search_response = requests.get(search_query_url)
    
    try:
        # Parse the search response JSON
        search_results = search_response.json()
        if search_results['results']:
            # Get the movie ID from the first result
            movie_id = search_results['results'][0]['id']
            
            # Construct the movie details query URL
            movie_query_url = f"{tmdb_movie_url}{movie_id}?api_key={tmdb_api_key}"
            
            # Make a GET request to retrieve the movie details
            movie_response = requests.get(movie_query_url)
            
            # Parse the movie details response JSON
            movie_details = movie_response.json()
            
            # Extract required fields
            movie_data = {
                "title": title,
                "original_title": movie_details.get("original_title"),
                "budget": movie_details.get("budget"),
                "original_language": movie_details.get("original_language"),
                "homepage": movie_details.get("homepage"),
                "overview": movie_details.get("overview"),
                "popularity": movie_details.get("popularity"),
                "runtime": movie_details.get("runtime"),
                "revenue": movie_details.get("revenue"),
                "release_date": movie_details.get("release_date"),
                "vote_average": movie_details.get("vote_average"),
                "vote_count": movie_details.get("vote_count"),
                "genres": [genre['name'] for genre in movie_details.get("genres", [])],
                "spoken_languages": [lang['english_name'] for lang in movie_details.get("spoken_languages", [])],
                "production_countries": [country['name'] for country in movie_details.get("production_countries", [])]
            }
            
            # Append the movie data to the tmdb_movies_list
            tmdb_movies_list.append(movie_data)
            print(f"Title '{title}' found and details retrieved.")

             # Increment the request counter
        request_counter += 1
        
        # Use time.sleep(1) when the request counter reaches a multiple of 50
        if request_counter % 50 == 0:
            print("Sleeping for 1 second to avoid hitting rate limits...")
            time.sleep(1)
    
    except Exception as e:
        print(f"Failed to get details for title '{title}': {e}")

# Preview the first five results
print(json.dumps(tmdb_movies_list[:5], indent=4))

# Convert the results to a DataFrame
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Display the DataFrame
df_tmdb_movies.head()

Title 'The Attachment Diaries' found and details retrieved.
Title 'You Can Live Forever' found and details retrieved.
Title 'A Tourist’s Guide to Love' found and details retrieved.
Title 'Other People’s Children' found and details retrieved.
Title 'One True Loves' found and details retrieved.
Title 'The Lost Weekend: A Love Story' found and details retrieved.
Title 'A Thousand and One' found and details retrieved.
Title 'Your Place or Mine' found and details retrieved.
Title 'Love in the Time of Fentanyl' found and details retrieved.
Title 'Pamela, a Love Story' found and details retrieved.
Sleeping for 1 second to avoid hitting rate limits...
Title 'In From the Side' found and details retrieved.
Title 'After Love' found and details retrieved.
Title 'Alcarràs' found and details retrieved.
Title 'Nelly & Nadine' found and details retrieved.
Title 'Lady Chatterley’s Lover' found and details retrieved.
Title 'The Sound of Christmas' found and details retrieved.
Title 'The Inspection' foun

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",12.581,96,0,2023-04-21,6.289,161,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",10.518,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"


In [16]:
#Part 3: Merge and Clean the Data for Export***********************************************

# Merge the New York Times reviews and TMDB DataFrames on the title column
df_merged = pd.merge(df_reviews, df_tmdb_movies, on='title')


In [17]:
# The genres, spoken_languages, and production_countries columns were saved as lists,
# but we want the columns to be strings without the list characters ([, ], and ').
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']
characters_to_remove = ["[", "]", "'"]

for column in columns_to_fix:
    df_merged[column] = df_merged[column].astype(str)
    for char in characters_to_remove:
        df_merged[column] = df_merged[column].str.replace(char, '')

# Print the head of the updated DataFrame to confirm the list characters were removed
print(df_merged.head())

                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/25/movies/the-...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/05/04/movies/you-...   
4  https://www.nytimes.com/2023/04/21/movies/a-to...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  A gynecologist and her patient form a horrifyi...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Religion comes between two girls falling in lo...  The New York Times   
4  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   

                                            keywords  \
0  Movies, The Attachment Diaries (Movie), Diment...   
1  Movies, The Attachment Diaries (Movie), Diment...   
2  Movies, You Can Live Forever (Movie

In [18]:
if 'byline.person' in df_merged.columns:
    df_merged.drop(columns=['byline.person'], inplace=True)

df_merged.drop_duplicates(inplace=True)
df_merged.reset_index(drop=True, inplace=True)



In [19]:
import os

# Create the output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Merge the New York Times reviews and TMDB DataFrames on the title column
df_merged = pd.merge(df_reviews, df_tmdb_movies, on='title')

# The genres, spoken_languages, and production_countries columns were saved as lists,
# but we want the columns to be strings without the list characters ([, ], and ').
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']
characters_to_remove = ["[", "]", "'"]

for column in columns_to_fix:
    df_merged[column] = df_merged[column].astype(str)
    for char in characters_to_remove:
        df_merged[column] = df_merged[column].str.replace(char, '')

# Print the head of the updated DataFrame to confirm the list characters were removed
print(df_merged.head())

# Drop the byline.person column if it exists (depending on the structure of your DataFrame)
if 'byline.person' in df_merged.columns:
    df_merged.drop(columns=['byline.person'], inplace=True)

# Delete any duplicate rows
df_merged.drop_duplicates(inplace=True)

# Reset the index
df_merged.reset_index(drop=True, inplace=True)

# Export data to a CSV file without the DataFrame's index
df_merged.to_csv('output/merged_movie_data.csv', index=False)

# Display the final DataFrame
df_merged.head()


                                             web_url  \
0  https://www.nytimes.com/2023/05/25/movies/the-...   
1  https://www.nytimes.com/2023/05/25/movies/the-...   
2  https://www.nytimes.com/2023/05/04/movies/you-...   
3  https://www.nytimes.com/2023/05/04/movies/you-...   
4  https://www.nytimes.com/2023/04/21/movies/a-to...   

                                             snippet              source  \
0  A gynecologist and her patient form a horrifyi...  The New York Times   
1  A gynecologist and her patient form a horrifyi...  The New York Times   
2  Religion comes between two girls falling in lo...  The New York Times   
3  Religion comes between two girls falling in lo...  The New York Times   
4  Rachael Leigh Cook stars in this bland rom-com...  The New York Times   

                                            keywords  \
0  Movies, The Attachment Diaries (Movie), Diment...   
1  Movies, The Attachment Diaries (Movie), Diment...   
2  Movies, You Can Live Forever (Movie

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"Movies, The Attachment Diaries (Movie), Diment...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"Movies, You Can Live Forever (Movie), Slutsky,...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"Movies, A Tourist's Guide to Love (Movie), Tsu...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",12.581,96,0,2023-04-21,6.289,161,"Romance, Comedy","English, Vietnamese",United States of America
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"Movies, Zlotowski, Rebecca, Other People's Chi...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",10.518,104,84178,2022-09-21,6.818,190,"Drama, Comedy","French, English",France
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"Movies, Bracey, Luke (1989- ), Liu, Simu, Reid...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
